In [ ]:
#%matplotlib notebook
%matplotlib inline
#%config InlineBackend.figure_formats=['svg']
#%config InlineBackend.figure_formats=['pdf']

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 9)

from matplotlib.colors import LogNorm

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

## NPE

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"

WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
#WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0       # 0 for gamma, 1 for protons
MIN_NPE = 0    # cut on SUM_PE_CLEANED_IMAGE

XLINE = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]
tc = tc[tc.peSum >= MIN_NPE]
#tc = tc[tc.border > 0]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]
wt = wt[wt.peSum >= MIN_NPE]
#wt = wt[wt.border > 0]

tc_wt = pd.merge(tc, wt, on="Id", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

In [ ]:
#df.groupby(["Type","Part","Success"]).count()

### NPE on cleaned image

In [ ]:
fig, ax = plt.subplots()
tc['peSum'].plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="blue",
                      label="Tailcut ({})".format(len(tc['peSum'])),  legend=True)
ref['peSum'].plot.hist(alpha=0.2, bottom=0.1, bins=np.logspace(0.1, 6., 90), color="green",
                       label="Reference ({})".format(len(ref['peSum'])), legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlabel("Number of photoelectrons", fontsize=20)
ax.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax.set_xscale('log')

ax.legend(prop={'size': 18})

In [ ]:
fig, ax = plt.subplots()
wt['peSum'].plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="red",
                      label="Wavelets ({})".format(len(wt['peSum'])),  legend=True)
ref['peSum'].plot.hist(alpha=0.2, bottom=0.1, bins=np.logspace(0.1, 6., 90), color="green",
                       label="Reference ({})".format(len(ref['peSum'])), legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlabel("Number of photoelectrons", fontsize=20)
ax.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax.set_xscale('log')

ax.legend(prop={'size': 18})

In [ ]:
fig, ax = plt.subplots()
wt['peSum'].plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="red",
                      label="Wavelets ({})".format(len(wt['peSum'])),  legend=True)
tc['peSum'].plot.hist(alpha=0.2,  bottom=0.1, bins=np.logspace(0.1, 6., 90), color="blue",
                      label="Tailcut ({})".format(len(tc['peSum'])),  legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlabel("Number of photoelectrons", fontsize=20)
ax.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax.set_xscale('log')

ax.legend(prop={'size': 18})

## Ratio

In [ ]:
def plot_ratio(ax, res_tuple_tc, res_tuple_wt):
    val_of_bins_tc, bins_tc, patches_tc = res_tuple_tc
    val_of_bins_wt, bins_wt, patches_wt = res_tuple_wt
    edges_of_bins = bins_tc

    # Set ratio where val_of_bins_data is not zero
    ratio = np.divide(val_of_bins_wt,
                      val_of_bins_tc,
                      where=(val_of_bins_tc != 0))

    # Compute error on ratio (null if cannot be computed)
    # This is wrong as it's made for Gaussian distributions and here we have Poisson distribution
    error = np.divide(val_of_bins_wt * np.sqrt(val_of_bins_tc) + val_of_bins_tc * np.sqrt(val_of_bins_wt),
                       np.power(val_of_bins_tc, 2),
                       where=(val_of_bins_tc != 0))

    ax.set_ylabel('Ratio (WT/TC)', fontsize=20)
    ax.axhline(y=1, linewidth=2, linestyle='--', color='gray', alpha=0.5)

    bincenter = 0.5 * (edges_of_bins[1:] + edges_of_bins[:-1])
    ax.errorbar(bincenter, ratio, yerr=error, fmt='o', color='k', elinewidth=3, capsize=4, capthick=3, linewidth=6)
    ax.plot(bincenter, ratio, 'ok', linewidth=6)
    
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 9))

# AX1

#wt['peSum'].plot.hist(bottom=0.1, legend=True)
BINS = np.logspace(0.1, 6., 30)


res_tuple_tc1 = ax1.hist(tc['peSum'],
                        label="Tailcut ({})".format(len(tc['peSum'])),
                        bins=BINS,
                        linewidth=2, alpha=0.5, color='blue',  histtype="step")

res_tuple_wt1 = ax1.hist(wt['peSum'],
                        label="Wavelets ({})".format(len(wt['peSum'])),
                        bins=BINS,
                        linewidth=2, alpha=0.5, color='red', histtype="step")

ax1.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax1.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax1.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax1.set_xlabel("Number of photoelectrons", fontsize=20)
ax1.set_ylabel("counts", fontsize=20)

#ax.set_yscale('log')
ax1.set_xscale('log')

ax1.legend(prop={'size': 18})

# AX2

plot_ratio(ax2, res_tuple_tc1, res_tuple_wt1)
ax2.set_xscale('log')
ax2.set_title("WT/TC")

ax2.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax2.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax2.axvline(x=XLINE, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax2.set_xlabel("Number of photoelectrons", fontsize=20)

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['peSum'], df['peSum_tc'], '.b', alpha=0.1, label="tc")
ax.plot(df['peSum'], df['peSum_wt'], '.r', alpha=0.1, label="wt")

ax.plot([0,100000], [0,100000], '--k', alpha=0.5, linewidth=2)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("npe ref", fontsize=20)
ax.set_ylabel("npe clean", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

## PE Sum difference per image

In [ ]:
df['peSum_wt-tc'] = df['peSum_wt'] - df['peSum_tc']

dfn = df[df["peSum_wt-tc"] < 0]
dfp = df[df["peSum_wt-tc"] >= 0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

ax1.plot(np.abs(dfn["peSum_wt-tc"]), dfn['peSum'], ".r",
         alpha=0.1, label="WT - TC (per image) < 0 ({})".format(len(dfn)))
ax2.plot(dfp["peSum_wt-tc"], dfp['peSum'], ".r",
         alpha=0.1, label="WT - TC (per image) > 0 ({})".format(len(dfp)))

fig.suptitle("WT - TC (per image)", fontsize=20)

ax1.set_yscale('log')
ax1.set_xscale('log')

ax2.set_yscale('log')
ax2.set_xscale('log')

ax1.set_xlabel("Sum PE (negative differences)", fontsize=20)
ax1.set_ylabel("Sum PE ref.", fontsize=20)

ax2.set_xlabel("Sum PE (positive differences)", fontsize=20)
ax2.set_ylabel("Sum PE ref.", fontsize=20)

ax1.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)
ax2.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

x1 = np.abs(dfn["peSum_wt-tc"])
x2 = dfp["peSum_wt-tc"]
y1 = dfn['peSum']
y2 = dfp['peSum']

xmin1 = np.log10(x1.min())
xmax1 = np.log10(x1.max())
ymin1 = np.log10(y1.min())
ymax1 = np.log10(y1.max())

xmin2 = np.log10(x2.min())
xmax2 = np.log10(x2.max())
ymin2 = np.log10(y2.min())
ymax2 = np.log10(y2.max())

xbins1 = np.logspace(xmin1, xmax1, 70) # <- make a range from 10**xmin to 10**xmax
ybins1 = np.logspace(ymin1, ymax1, 70) # <- make a range from 10**ymin to 10**ymax

xbins2 = np.logspace(xmin2, xmax2, 70) # <- make a range from 10**xmin to 10**xmax
ybins2 = np.logspace(ymin2, ymax2, 70) # <- make a range from 10**ymin to 10**ymax

counts1, _, _ = np.histogram2d(x1, y1, bins=(xbins1, ybins1))
pcm1 = ax1.pcolormesh(xbins1, ybins1, counts1, norm=LogNorm())
fig.colorbar(pcm1, ax=ax1)

counts2, _, _ = np.histogram2d(x2, y2, bins=(xbins2, ybins2))
pcm2 = ax2.pcolormesh(xbins2, ybins2, counts2, norm=LogNorm())
fig.colorbar(pcm2, ax=ax2)

#im1 = ax1.hexbin(x1, y1, gridsize=40)
#im2 = ax2.hexbin(x2, y2, gridsize=40)

#fig.colorbar(im1, ax=ax1)
#fig.colorbar(im2, ax=ax2)

fig.suptitle("WT - TC (per image)", fontsize=20)

ax2.set_xlim(0.0001, 1000)
ax2.set_ylim(1, 10000)

ax1.set_title("WT - TC (per image) < 0 ({})".format(len(dfn)))
ax2.set_title("WT - TC (per image) > 0 ({})".format(len(dfp)))

ax1.set_yscale('log')
ax1.set_xscale('log')

ax2.set_yscale('log')
ax2.set_xscale('log')

ax1.set_xlabel("Sum PE (negative differences)", fontsize=20)
ax1.set_ylabel("Sum PE ref.", fontsize=20)

ax2.set_xlabel("Sum PE (positive differences)", fontsize=20)
ax2.set_ylabel("Sum PE ref.", fontsize=20)

ax1.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)
ax2.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

## PE sum TC VS PE sum WT

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

ax.plot(df['peSum_wt'], df['peSum_tc'], ".r", alpha=0.1)

ax.plot([0,100000], [0,100000], '--k', alpha=0.5, linewidth=2)

ax.set_title("PE sum TC VS PE sum WT", fontsize=20)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim(1, 100000)
ax.set_ylim(10, 100000)

ax.set_xlabel("Sum PE WT", fontsize=20)
ax.set_ylabel("Sum PE TC", fontsize=20)